In [1]:
5+5

10

In [17]:
GPT_CONFIG_124M = {
    "vocab_size": 22000,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

In [18]:
import torch
import torch.nn as nn









In [19]:
class DummyTransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        # A simple placeholder

    def forward(self, x):
        # This block does nothing and just returns its input.
        return x

In [20]:
class DummyLayerNorm(nn.Module):
    def __init__(self, normalized_shape, eps=1e-5):
        super().__init__()
        # The parameters here are just to mimic the LayerNorm interface.

    def forward(self, x):
        # This layer does nothing and just returns its input.
        return x

In [21]:

class DummyGPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        
        # Use a placeholder for TransformerBlock
        self.trf_blocks = nn.Sequential(
            *[DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])])
        
        # Use a placeholder for LayerNorm
        self.final_norm = DummyLayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

In [22]:
import os
import sentencepiece as spm



tokenizer_path = os.path.join("..", "tokenizer_model", "hadith_unigram")

sp = spm.SentencePieceProcessor()
sp.load(tokenizer_path + ".model" )

batch = []

txt1 = "السلام عليكم ورحمه "
txt2 = "قال الامام "

batch.append(torch.tensor(sp.encode_as_ids(txt1)))
batch.append(torch.tensor(sp.encode_as_ids(txt2)))
batch = torch.stack(batch, dim=0)
print(batch)

tensor([[ 222,  331,   16, 9154],
        [  10,  401,   94,   47]])


In [23]:
torch.manual_seed(123)
model = DummyGPTModel(GPT_CONFIG_124M)

logits = model(batch)
print("Output shape:", logits.shape)
print(logits)

Output shape: torch.Size([2, 4, 22000])
tensor([[[-1.4276,  0.7188,  0.0363,  ...,  1.4750, -0.8175, -0.5635],
         [-1.6214,  0.0083,  2.1138,  ...,  1.7436, -1.1488,  0.6183],
         [ 0.8778, -1.5745,  0.4970,  ...,  0.6816, -0.9175,  0.6595],
         [-0.0464, -0.5917, -0.0514,  ..., -0.0279, -1.2905, -0.1687]],

        [[-0.2413,  1.3572,  0.5811,  ...,  0.8343, -1.9712, -0.4528],
         [-0.9282, -1.1979,  0.2855,  ...,  1.2372, -0.3929,  0.3227],
         [-0.1779, -1.2816, -1.5905,  ..., -0.3224, -0.4690,  1.4036],
         [ 0.8358,  0.3216,  1.5397,  ..., -0.4481, -0.0552,  1.2528]]],
       grad_fn=<UnsafeViewBackward0>)
